In [2]:
# importing the libraries
import urllib.request    
from time import sleep
from random import uniform
import re,time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd

In [6]:
products=[]
products.append('https://www.amazon.com/Dell-Latitude-E5470-Certified-Refurbished/product-reviews/B07D5N1PFY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

In [ ]:
#!pip install requests_html
driver = webdriver.Chrome('/webscraping/chromedriver')
def getHtmlSource(url):
    driver.get(url)
    page_source = driver.page_source
    return bs(page_source,'lxml') #html parser

In [9]:
df = pd.DataFrame(columns=['id','title','date','rating','helpful','review'])
df.head()

,id,title,date,rating,helpful,review


In [10]:
def getReviews(url,tag,filtertype,class_filter,reviews_list):
    if len(reviews_list)>=10:
        print("max length reached")
        print(url)
        return reviews_list
    driver.get(url)
    products_page_source = driver.page_source
    products_html_source = bs(products_page_source,'lxml') #html parserpage_source = driver.page_source
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    div_tags = products_html_source.find_all('div', attrs={'class':'a-section review aok-relative'})
    for div in div_tags:
        div_id=div['id']
        try:
            for child in div:
                if child.find('span', attrs={'class': 'reviews-loading'}):
                    break
                title=child.find('a', attrs={'data-hook':'review-title'}).span.text
                date=child.find('span', attrs={'data-hook':'review-date'}).text
                rating=child.find('i', attrs={'data-hook':'review-star-rating'}).span.text
                helpful=child.find('span', attrs={'data-hook':'helpful-vote-statement'}).text
                review=child.find('span', attrs={'class':'a-size-base review-text review-text-content'}).span.text
                new_row = {'id':div_id, 'title':title, 'date':date, 'rating':rating, 'helpful':helpful, 'review':review}
                reviews_list.append(new_row)
        except:
            continue
    next_page=products_html_source.find_all('li', attrs={'class':'a-last'})
    #print("next_page",next_page)
    if len(next_page)>0:
        try:
            a=str(next_page[0])
            a=a[a.index('href="')+6:a.index('">Next page')]
            url= "https://amazon.com"+a.replace("amp;","")
            getReviews(url,'div','class','a-section review aok-relative',reviews_list)
        except:
            return reviews_list
    else:
        print('len of nextpage not zero')
        return  reviews_list     

In [15]:
def saveCsv(reviews_list):
    df = pd.DataFrame(reviews_list,columns=['id','title','date','rating','helpful','review'])
    df.to_csv("sample.csv")

In [20]:
for product in products:
    reviews_list=[]
    getReviews(url,'div','class','a-section review aok-relative',reviews_list)
    print(len(reviews_list))
    saveCsv(reviews_list)

max length reached
https://amazon.com/Dell-Latitude-E5470-Certified-Refurbished/product-reviews/B07D5N1PFY/ref=cm_cr_arp_d_paging_btm_3?ie=UTF8&pageNumber=3&reviewerType=all_reviews
12


In [21]:
reviews_list

[{'id': 'R205AWG8UMOSD6',
  'title': 'It will do.',
  'date': 'Reviewed in the United States 🇺🇸 on September 23, 2022',
  'rating': '3.0 out of 5 stars',
  'helpful': '6 people found this helpful',
  'review': "I bought this for school and I just hope I don't burn my house down with it. The cord and brick get very hot. I actually burnt my hand picking it up. Not sure if the touch screen is a thing or not, doesn't seem to be. And the battery life is short and unpredictable. Other than that it's fine. I'm not going to be buying from this store again. The payment process was a mess and shipping kept getting delayed. Then for some reason they sent it from Florida, to New York, only to come back down to Tennessee. Took forever and I needed it for the beginning of school. All in all I'd just have to say don't do it."},
 {'id': 'R2OA7GIJL752HW',
  'title': 'Battery died within a few hours',
  'date': 'Reviewed in the United States 🇺🇸 on November 10, 2022',
  'rating': '3.0 out of 5 stars',
  